# Exercise parser

In [1]:
# %pip install texsoup

In [2]:
import os
import TexSoup
import re

In [3]:
soup = TexSoup.TexSoup("""
\begin{document}

Blah $\sin\theta\sfT$, fah.

\[
 x = y^\sfT
\]


\end{document}
""")

In [4]:
for idx, el in enumerate(soup.contents):
    if hasattr(el, "name"):
        name = el.name
    else:
        name = ""
    print(idx, "----", type(el), name)
    print(el)

0 ---- <class 'TexSoup.utils.Token'> 

egin
1 ---- <class 'TexSoup.data.TexNode'> BraceGroup
{document}
2 ---- <class 'TexSoup.utils.Token'> 

Blah 
3 ---- <class 'TexSoup.data.TexNode'> $
$\sin	heta\sfT$
4 ---- <class 'TexSoup.utils.Token'> 
, fah.


5 ---- <class 'TexSoup.data.TexNode'> displaymath
\[
 x = y^\sfT
\]
6 ---- <class 'TexSoup.data.TexNode'> end
\end{document}


In [5]:
# DIR = "/Users/ivan/Projects/Minireference/STATSbook/statsprobs/"
# FILENAMES = [
#     "chapter31_1_estimators.tex",
#     "chapter31_2_sampling_dists.tex",
#     "chapter31_3_approximations.tex",
#     "chapter31_4_mean.tex",
#     "chapter31_5_variance.tex",
#     "chapter31_6_dmeans.tex",
#     "chapter31_7_final.tex",
#     "chapter32_confidence_intervals.tex",
# ]

In [6]:
DIR = "/Users/ivan/Projects/Minireference/LAbook/noBSLAexercises"
FILENAMES = [
    "chapter2_exercises_01_introduction.tex",
    "chapter2_exercises_02_vector_operations.tex",
    "chapter2_exercises_03_matrix_operations.tex",
    "chapter2_exercises_04_linearity.tex",
]

In [7]:
filename = FILENAMES[3]
filepath = os.path.join(DIR, filename)

In [8]:
texsrc = open(filepath).read()
soup = TexSoup.TexSoup(texsrc)

In [9]:
exercises = soup.exercises.find_all("exercise")
# for exercise in exercises:
#     print(exercise)

In [10]:
exercise = exercises[0]
# exercise.find("ansparts*")

for idx, el in enumerate(exercise.contents):
    print(idx, "----", type(el))
    print(el)

0 ---- <class 'TexSoup.utils.Token'>

		Are these expressions linear in the variables 
1 ---- <class 'TexSoup.data.TexNode'>
$x$
2 ---- <class 'TexSoup.utils.Token'>
, 
3 ---- <class 'TexSoup.data.TexNode'>
$y$
4 ---- <class 'TexSoup.utils.Token'>
, and 
5 ---- <class 'TexSoup.data.TexNode'>
$z$
6 ---- <class 'TexSoup.utils.Token'>
?

		
7 ---- <class 'TexSoup.data.TexNode'>
{\small
		\begin{exparts*}
			\partsitem $2x+5y + \sqrt{m}z$
			\partsitem $10\sqrt{x} + 2(y+z)$
			\partsitem $42x + \alpha^2\sin(\frac{\pi}{3})y + z\cos(\frac{\pi}{3})$
		\end{exparts*}
		}
8 ---- <class 'TexSoup.data.TexNode'>
\begin{eanswer}\begin{ansparts*}
					\partsitem Yes;
					\partsitem No;
					\partsitem Yes.
					\end{ansparts*}\end{eanswer}
9 ---- <class 'TexSoup.data.TexNode'>
\begin{esolution}
			An expression is linear in the variable $v$ if it contains $v$ raised only to the first power.
			This is the case for the first and third expressions but not the second,
			since it contains $\sqrt{x} = 

In [11]:
el =  exercise.contents[9]
el.contents

['\n\t\t\tAn expression is linear in the variable ',
 $v$,
 ' if it contains ',
 $v$,
 ' raised only to the first power.\n\t\t\tThis is the case for the first and third expressions but not the second,\n\t\t\tsince it contains ',
 $\sqrt{x} = x^{\frac{1}{2}}$,
 '.\n\t\t']

In [12]:
# lines = exercise.text
# outlines = []

# for line in lines:
#     line = line.strip()
#     if line.startswith("%"):
#         continue
#     else:
#         outlines.append(line)
#  outlines

In [13]:
chcounter = 2
seccounter = 1
ecounter = 1
partscounter = 1
letters = "_abcdefghijklmnopqrstuvwxyz"

COLUMNS_CMDS = ["twocol", "endtwocol", "threecol", "endthreecol", "fourcol", "endfourcol"]

MACRO_REPLACEMENTS = {
    r"\sfT": r"{\mathsf{T}}",
    r"\Tr": r"\textup{Tr}",
}

mdlines = []

for filename in FILENAMES:
    mdlines.append(f"## Exercises {seccounter}")
    mdlines.extend(["", ""])
    filepath = os.path.join(DIR, filename)
    texsrc = open(filepath).read()
    soup = TexSoup.TexSoup(texsrc)
    exercises = soup.exercises.find_all("exercise")
    for exercise in exercises:
        mdlines.append(f"### E{chcounter}.{ecounter}")
        mdlines.append("")
        firstline = True
        # print(exercise.string[0:4])
        contents = ""
        elements = exercise.contents
        while elements:
            el = elements.pop(0)
            if type(el) == TexSoup.utils.Token:
                el = el.replace("\t", "")
                text = re.sub(r"\s+", " ", el)
                text = text.replace("~", "")
                text = text.replace(r"\;", "")
                if text.startswith("%"):
                    continue
                if firstline:
                    contents += text.lstrip()
                else:
                    contents += text
                firstline = False
            elif type(el) == TexSoup.data.TexNode:
                if hasattr(el, "name") and el.name == "tt":
                    contents += " `" + el.string + "` "
                elif hasattr(el, "name") and el.name == "textbf":
                    contents += " **" + el.string + "** "
                elif hasattr(el, "name") and el.name == "partsitem":
                    label = letters[partscounter]
                    contents += "\n - " + label + ") "
                    partscounter += 1
                elif hasattr(el, "name") and el.name in COLUMNS_CMDS:
                    contents += "\n"
                elif hasattr(el, "name") and el.name in ["exparts", "exparts*", "ansparts", "ansparts*"]:
                    partscounter = 1
                    elements = el.contents + elements
                    contents += "\n"
                elif hasattr(el, "name") and el.name in ["eanswer", "answer"]:
                    contents += "\n\n#### Answer\n"
                    firstline = True
                    elements = el.contents + elements
                elif hasattr(el, "name") and el.name in ["esolution", "solution"]:
                    contents += "\n\n#### Solution\n"
                    firstline = True
                    elements = el.contents + elements
                elif hasattr(el, "name") and el.name == "hint":
                    contents += "\n\n**Hint**: "
                    elements = el.contents + elements
                elif hasattr(el, "name") and el.name in ["$", "displaymath"]:
                    elstr = str(el)
                    elstr = elstr.replace("\t", "")
                    for key, value in MACRO_REPLACEMENTS.items():
                        elstr = elstr.replace(key, value)
                    contents += elstr
                else:
                    elstr = str(el)
                    elstr = elstr.replace("\t", "")
                    contents += elstr
        mdlines.append(contents)
        mdlines.extend(["", ""])
        ecounter += 1
    seccounter += 1

        
print("\n".join(mdlines))

## Exercises 1


### E2.1

Find the inverse matrix $A^{-1}$ for the matrix $A=\begin{bsmallmatrix}7 & 0 \\ 0 & 2\end{bsmallmatrix}$. Verify that $A^{-1}(A\vec{v})=\vec{v}$ for any vector $\vec{v} = \begin{bsmallmatrix} v_1 \\ v_2\end{bsmallmatrix}$. 

#### Answer
$A^{-1} = \begin{bsmallmatrix}\frac{1}{7} & 0 \\ 0 & \frac{1}{2}\end{bsmallmatrix}$.

#### Solution
To find $A^{-1}$ we must consider the action of $A=\begin{bsmallmatrix}7 & 0 \\ 0 & 2\end{bsmallmatrix}$ on an arbitrary vector $\vec{v}=\begin{bsmallmatrix}v_1 \\ v_2\end{bsmallmatrix}$, and perform the inverse action. Since $A$ multiplies the first component by $7$, $A^{-1}$ must divide the first component by $7$. Since $A$ multiplies the second component by $2$, $A^{-1}$ must divide the second component by $2$. Thus $A^{-1} = \begin{bsmallmatrix}\frac{1}{7} & 0 \\ 0 & \frac{1}{2}\end{bsmallmatrix}$. 


### E2.2

Given the matrices $A=\begin{bsmallmatrix}1 & 3 \\ 4 & 5\end{bsmallmatrix}$ and $B=\begin{bsmallmatrix} -1 & 0 \\ 3